In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [3]:
from sklearn.datasets import load_iris

In [4]:
X = pd.DataFrame(load_iris().data,columns=load_iris().feature_names)

In [5]:
Y = pd.DataFrame(load_iris().target,columns=['target'])

In [6]:
df = pd.DataFrame(pd.concat([X,Y],axis=1))

In [11]:
df[df.duplicated()==True]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
142,5.8,2.7,5.1,1.9,2


In [13]:
# profile_report.to_widgets()
# profile_report.to_notebook_iframe()

profile_report = ProfileReport(df,title="Automated EDA",explorative=True)
profile_report.to_widgets()

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [16]:
std_scaler = StandardScaler()
scaled_dataset = std_scaler.fit_transform(df.iloc[:,:-1])

In [17]:
rf_clf = RandomForestClassifier(n_jobs=-1,random_state=1)
log_reg_clf = LogisticRegression(random_state=2,n_jobs=-1)
gauss_clf = GaussianNB()

model_name = ['Logistic','Gaussian NB','Random Forest']

model_zipped = zip(model_name,[log_reg_clf,gauss_clf,rf_clf])

In [18]:
for model_name, model in model_zipped:
    scores = model_selection.cross_val_score(model,scaled_dataset,df.iloc[:,-1],cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]"% (scores.mean(), scores.std(), model_name))

Accuracy: 0.95 (+/- 0.03) [Logistic]
Accuracy: 0.95 (+/- 0.03) [Gaussian NB]
Accuracy: 0.97 (+/- 0.02) [Random Forest]


In [19]:
sft_voting_clf = VotingClassifier(estimators=[(model_name, model) for model_name,model in model_zipped],voting='soft')
hrd_voting_clf = VotingClassifier(estimators=[(model_name, model) for model_name,model in model_zipped],voting='hard')

In [20]:
sft_voting_clf_scores = model_selection.cross_val_score(sft_voting_clf,scaled_dataset,df.iloc[:,-1],cv=5, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]"% (scores.mean(), scores.std(),"Soft Voting"))

Accuracy: 0.97 (+/- 0.02) [Soft Voting]


In [21]:
hrd_voting_clf_scores = model_selection.cross_val_score(hrd_voting_clf,scaled_dataset,df.iloc[:,-1],cv=5, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]"% (scores.mean(), scores.std(),"Hard Voting"))

Accuracy: 0.97 (+/- 0.02) [Hard Voting]
